# 🚀 Missile PID Tuning with RL - Kaggle GPU Training

Train reinforcement learning agents (PPO, SAC, TD3) to adaptively tune PID parameters for missile guidance.

**Requirements:**
- Kaggle GPU (T4 or P100)
- Internet enabled (for pip install)

**Training Time:**
- PPO: ~30-45 minutes (1M timesteps)
- SAC: ~45-60 minutes (1M timesteps)
- TD3: ~45-60 minutes (1M timesteps)

## 1. Setup Environment

In [ ]:
%%time
# Install dependencies with fixed versions
# NumPy 2.x causes compatibility issues, pin to 1.x
!pip install -q 'numpy<2' gymnasium 'stable-baselines3[extra]' torch tensorboard

print("\n✓ Dependencies installed")

In [ ]:
# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be slow.")

## 2. Define Missile PID System

In [ ]:
import numpy as np

class PIDController:
    """Simple PID controller"""
    def __init__(self, kp=2.0, ki=0.1, kd=0.5):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral = 0.0
        self.prev_error = 0.0

    def compute(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        output = self.kp * error + self.ki * self.integral + self.kd * derivative
        self.prev_error = error
        return output

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0

    def set_gains(self, kp, ki, kd):
        self.kp = kp
        self.ki = ki
        self.kd = kd


class Missile:
    """2D Missile with PID guidance - HIGH SPEED VERSION"""
    def __init__(self, x=0.0, y=0.0, vx=800.0, vy=0.0,
                 max_speed=1000.0, max_accel=1000.0,
                 kp=2.0, ki=0.1, kd=0.5):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.max_speed = max_speed
        self.max_accel = max_accel
        self.pid = PIDController(kp, ki, kd)
        self.trajectory = [(x, y)]
        self.active = True
        self.fuel = 1.0

    @property
    def position(self):
        return np.array([self.x, self.y])

    @property
    def velocity(self):
        return np.array([self.vx, self.vy])

    @property
    def speed(self):
        return np.linalg.norm(self.velocity)

    @property
    def heading(self):
        return np.arctan2(self.vy, self.vx)

    def update(self, target_pos, dt):
        if not self.active:
            return

        dx = target_pos[0] - self.x
        dy = target_pos[1] - self.y
        desired_heading = np.arctan2(dy, dx)
        error = desired_heading - self.heading
        error = np.arctan2(np.sin(error), np.cos(error))

        control = self.pid.compute(error, dt)
        control = np.clip(control, -self.max_accel, self.max_accel)

        if self.speed > 0:
            perp = np.array([-self.vy, self.vx]) / self.speed
            self.vx += perp[0] * control * dt
            self.vy += perp[1] * control * dt

        current_speed = self.speed
        if current_speed > 0:
            scale = self.max_speed / current_speed
            self.vx *= scale
            self.vy *= scale

        self.x += self.vx * dt
        self.y += self.vy * dt
        self.fuel -= 0.01 * dt
        
        if self.fuel <= 0:
            self.active = False

        self.trajectory.append((self.x, self.y))

    def reset(self, x=0.0, y=0.0, vx=800.0, vy=0.0):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.active = True
        self.fuel = 1.0
        self.trajectory = [(x, y)]
        self.pid.reset()

    def set_pid_gains(self, kp, ki, kd):
        self.pid.set_gains(kp, ki, kd)

print("✓ Missile system defined (HIGH SPEED: 1000 m/s, 1000 m/s²)")

## 3. Define Moving Target

In [ ]:
class Target:
    """Moving target with different maneuver patterns - HIGH SPEED VERSION"""
    def __init__(self, x=8000.0, y=5000.0, speed=1000.0, maneuver='straight'):
        self.x = x
        self.y = y
        self.speed = speed
        self.heading = np.pi
        self.maneuver = maneuver
        self.time = 0.0
        self.trajectory = [(x, y)]

    @property
    def position(self):
        return np.array([self.x, self.y])

    @property
    def velocity(self):
        vx = self.speed * np.cos(self.heading)
        vy = self.speed * np.sin(self.heading)
        return np.array([vx, vy])

    def update(self, dt, missile_pos=None):
        self.time += dt

        if self.maneuver == 'straight':
            pass
        elif self.maneuver == 'circular':
            turn_rate = 0.3
            self.heading += turn_rate * dt
        elif self.maneuver == 'zigzag':
            if int(self.time) % 4 < 2:
                turn_rate = 0.5
            else:
                turn_rate = -0.5
            self.heading += turn_rate * dt
        elif self.maneuver == 'evasive' and missile_pos is not None:
            dx = self.x - missile_pos[0]
            dy = self.y - missile_pos[1]
            distance = np.sqrt(dx**2 + dy**2)
            if distance < 3000:
                escape_heading = np.arctan2(dy, dx)
                heading_diff = escape_heading - self.heading
                heading_diff = np.arctan2(np.sin(heading_diff), np.cos(heading_diff))
                turn_rate = 0.8 * np.sign(heading_diff)
                self.heading += turn_rate * dt

        vx = self.speed * np.cos(self.heading)
        vy = self.speed * np.sin(self.heading)
        self.x += vx * dt
        self.y += vy * dt
        self.trajectory.append((self.x, self.y))

    def reset(self, x=8000.0, y=5000.0, heading=np.pi):
        self.x = x
        self.y = y
        self.heading = heading
        self.time = 0.0
        self.trajectory = [(x, y)]

print("✓ Target system defined (HIGH SPEED: 1000 m/s)")

## 4. Define Gymnasium Environment

In [ ]:
import gymnasium as gym
from gymnasium import spaces

class MissilePIDEnv(gym.Env):
    """Gym environment for missile PID tuning - HIGH SPEED VERSION"""
    metadata = {'render_modes': ['human', 'rgb_array'], 'render_fps': 60}

    def __init__(self, map_size=10000.0, hit_radius=50.0, max_steps=500,
                 dt=0.01, target_maneuver='circular'):
        super().__init__()
        self.map_size = map_size
        self.hit_radius = hit_radius
        self.max_steps = max_steps
        self.dt = dt
        self.target_maneuver = target_maneuver

        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(3,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(14,), dtype=np.float32)

        self.kp_range = (0.1, 10.0)
        self.ki_range = (0.0, 5.0)
        self.kd_range = (0.0, 5.0)

        self.missile = None
        self.target = None
        self.step_count = 0
        self.prev_distance = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        missile_x = np.random.uniform(0, 2000)
        missile_y = np.random.uniform(2000, 8000)
        missile_vx = np.random.uniform(800, 900)  # Higher initial speed
        missile_vy = 0.0

        target_x = np.random.uniform(7000, 9000)
        target_y = np.random.uniform(2000, 8000)
        target_heading = np.random.uniform(0, 2*np.pi)

        self.missile = Missile(x=missile_x, y=missile_y, vx=missile_vx, vy=missile_vy,
                              max_speed=1000.0, max_accel=1000.0,
                              kp=2.0, ki=0.1, kd=0.5)
        self.target = Target(x=target_x, y=target_y, speed=1000.0, maneuver=self.target_maneuver)
        self.target.heading = target_heading

        self.step_count = 0
        self.prev_distance = np.linalg.norm(self.missile.position - self.target.position)

        return self._get_observation(), {}

    def step(self, action):
        self.step_count += 1

        delta_kp, delta_ki, delta_kd = action
        current_kp = self.missile.pid.kp
        current_ki = self.missile.pid.ki
        current_kd = self.missile.pid.kd

        new_kp = np.clip(current_kp + delta_kp * 0.5, self.kp_range[0], self.kp_range[1])
        new_ki = np.clip(current_ki + delta_ki * 0.2, self.ki_range[0], self.ki_range[1])
        new_kd = np.clip(current_kd + delta_kd * 0.2, self.kd_range[0], self.kd_range[1])

        self.missile.set_pid_gains(new_kp, new_ki, new_kd)
        self.missile.update(self.target.position, self.dt)
        self.target.update(self.dt, self.missile.position)

        distance = np.linalg.norm(self.missile.position - self.target.position)
        reward = -distance / self.map_size

        if self.prev_distance is not None:
            progress = (self.prev_distance - distance) / self.map_size
            reward += progress * 10.0
        self.prev_distance = distance

        terminated = False
        truncated = False

        if distance < self.hit_radius:
            reward += 100.0
            terminated = True

        if (not self.missile.active or
            self.missile.x < 0 or self.missile.x > self.map_size or
            self.missile.y < 0 or self.missile.y > self.map_size or
            self.target.x < 0 or self.target.x > self.map_size or
            self.target.y < 0 or self.target.y > self.map_size):
            reward -= 50.0
            terminated = True

        if self.step_count >= self.max_steps:
            truncated = True

        info = {
            'distance': distance,
            'hit': distance < self.hit_radius,
            'step': self.step_count,
            'pid_gains': {'kp': self.missile.pid.kp, 'ki': self.missile.pid.ki, 'kd': self.missile.pid.kd}
        }

        return self._get_observation(), reward, terminated, truncated, info

    def _get_observation(self):
        missile_pos = self.missile.position
        missile_vel = self.missile.velocity
        target_pos = self.target.position
        target_vel = self.target.velocity

        dx = target_pos[0] - missile_pos[0]
        dy = target_pos[1] - missile_pos[1]
        distance = np.sqrt(dx**2 + dy**2)

        desired_heading = np.arctan2(dy, dx)
        current_heading = self.missile.heading
        angle_error = desired_heading - current_heading
        angle_error = np.arctan2(np.sin(angle_error), np.cos(angle_error))

        obs = np.array([
            missile_pos[0] / self.map_size,
            missile_pos[1] / self.map_size,
            missile_vel[0] / self.missile.max_speed,
            missile_vel[1] / self.missile.max_speed,
            target_pos[0] / self.map_size,
            target_pos[1] / self.map_size,
            target_vel[0] / self.target.speed,
            target_vel[1] / self.target.speed,
            distance / self.map_size,
            angle_error / np.pi,
            self.missile.pid.kp / 10.0,
            self.missile.pid.ki / 5.0,
            self.missile.pid.kd / 5.0,
            self.missile.fuel
        ], dtype=np.float32)

        return obs

print("✓ Gymnasium environment defined (HIGH SPEED: 1000 m/s both)")

## 5. Configuration

In [ ]:
# Training configuration - HIGH SPEED + DEEP SAC
ALGORITHM = 'SAC'  # Deep Soft Actor-Critic
TARGET_MANEUVER = 'circular'  # Circular maneuver
TOTAL_TIMESTEPS = 1_000_000  # 1M timesteps
N_ENVS = 1  # Single environment

print(f"Configuration:")
print(f"  Algorithm: {ALGORITHM} (Deep RL)")
print(f"  Target Maneuver: {TARGET_MANEUVER}")
print(f"  Total Timesteps: {TOTAL_TIMESTEPS:,}")
print(f"  Environment: Single (DummyVecEnv)")
print(f"\n🚀 HIGH SPEED MODE:")
print(f"  Missile: 1000 m/s, 1000 m/s²")
print(f"  Target: 1000 m/s")
print(f"\n🧠 DEEP RL: 3-layer networks (256 units each)")

## 6. Create Training Environment

In [ ]:
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
import os
from datetime import datetime

def make_env(rank, maneuver):
    """Create environment instance"""
    def _init():
        return MissilePIDEnv(
            map_size=10000.0,
            hit_radius=50.0,
            max_steps=500,
            dt=0.01,
            target_maneuver=maneuver
        )
    return _init

# Create vectorized environments (use DummyVecEnv to avoid multiprocessing issues)
env = DummyVecEnv([make_env(0, TARGET_MANEUVER)])

# Create eval environment
eval_env = DummyVecEnv([make_env(0, TARGET_MANEUVER)])

print(f"✓ Training environment created")
print(f"✓ Using DummyVecEnv (single process, no multiprocessing issues)")

## 7. Initialize RL Algorithm

In [ ]:
# Create output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"{ALGORITHM}_{TARGET_MANEUVER}_{timestamp}"
run_dir = f"/kaggle/working/{run_name}"
os.makedirs(run_dir, exist_ok=True)

# Deep RL network architecture
# 3 hidden layers, 256 units each (deeper than default 2x256)
policy_kwargs = dict(
    net_arch=dict(
        pi=[256, 256, 256],  # Actor network: 3 layers
        qf=[256, 256, 256]   # Critic network: 3 layers
    )
)

# Initialize algorithm
if ALGORITHM == 'PPO':
    model = PPO(
        'MlpPolicy',
        env,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.01,
        verbose=1,
        tensorboard_log=os.path.join(run_dir, 'tensorboard'),
        device='cuda',
        policy_kwargs=policy_kwargs
    )
elif ALGORITHM == 'SAC':
    model = SAC(
        'MlpPolicy',
        env,
        learning_rate=3e-4,
        buffer_size=100_000,
        learning_starts=1000,
        batch_size=256,
        tau=0.005,
        gamma=0.99,
        train_freq=1,
        gradient_steps=1,
        ent_coef='auto',
        verbose=1,
        tensorboard_log=os.path.join(run_dir, 'tensorboard'),
        device='cuda',
        policy_kwargs=policy_kwargs  # Deep network
    )
elif ALGORITHM == 'TD3':
    model = TD3(
        'MlpPolicy',
        env,
        learning_rate=3e-4,
        buffer_size=100_000,
        learning_starts=1000,
        batch_size=256,
        tau=0.005,
        gamma=0.99,
        train_freq=1,
        gradient_steps=1,
        policy_delay=2,
        verbose=1,
        tensorboard_log=os.path.join(run_dir, 'tensorboard'),
        device='cuda',
        policy_kwargs=policy_kwargs
    )
else:
    raise ValueError(f"Unknown algorithm: {ALGORITHM}")

print(f"✓ {ALGORITHM} model initialized on GPU")
print(f"✓ Deep RL: 3-layer networks (256→256→256)")
print(f"✓ Output directory: {run_dir}")

## 8. Setup Callbacks

In [ ]:
# Checkpoint callback - save model periodically
checkpoint_callback = CheckpointCallback(
    save_freq=50_000 // N_ENVS,
    save_path=os.path.join(run_dir, 'checkpoints'),
    name_prefix='model'
)

# Evaluation callback - evaluate and save best model
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=os.path.join(run_dir, 'best_model'),
    log_path=os.path.join(run_dir, 'eval_logs'),
    eval_freq=10_000 // N_ENVS,
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

print("✓ Callbacks configured")

## 9. Train Model 🚀

This will take 30-60 minutes depending on GPU and algorithm.

In [ ]:
%%time
print(f"\n{'='*60}")
print(f"Training {ALGORITHM} on {TARGET_MANEUVER} target")
print(f"Total timesteps: {TOTAL_TIMESTEPS:,}")
print(f"{'='*60}\n")

# Train!
model.learn(
    total_timesteps=TOTAL_TIMESTEPS,
    callback=[checkpoint_callback, eval_callback],
    progress_bar=True
)

# Save final model
final_path = os.path.join(run_dir, 'final_model')
model.save(final_path)

print(f"\n{'='*60}")
print(f"✓ Training complete!")
print(f"✓ Final model saved: {final_path}")
print(f"✓ Best model saved: {os.path.join(run_dir, 'best_model')}")
print(f"{'='*60}")

## 10. Evaluate Trained Model

In [ ]:
# Load best model
best_model_path = os.path.join(run_dir, 'best_model', 'best_model.zip')

if ALGORITHM == 'PPO':
    eval_model = PPO.load(best_model_path)
elif ALGORITHM == 'SAC':
    eval_model = SAC.load(best_model_path)
elif ALGORITHM == 'TD3':
    eval_model = TD3.load(best_model_path)

# Evaluate
n_eval_episodes = 20
eval_env_single = MissilePIDEnv(
    map_size=10000.0,
    hit_radius=50.0,
    max_steps=500,
    dt=0.01,
    target_maneuver=TARGET_MANEUVER
)

hits = 0
total_rewards = []
distances = []
steps_list = []

print(f"\nEvaluating model on {n_eval_episodes} episodes...\n")
print("-" * 80)

for episode in range(n_eval_episodes):
    obs, info = eval_env_single.reset()
    episode_reward = 0
    done = False
    step = 0

    while not done:
        action, _ = eval_model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = eval_env_single.step(action)
        episode_reward += reward
        step += 1
        done = terminated or truncated

    total_rewards.append(episode_reward)
    distances.append(info['distance'])
    steps_list.append(step)

    if info['hit']:
        hits += 1
        status = "✓ HIT"
    else:
        status = "✗ MISS"

    print(f"Episode {episode+1:2d}: {status} | "
          f"Distance: {info['distance']:6.1f}m | "
          f"Reward: {episode_reward:7.1f} | "
          f"Steps: {step:3d} | "
          f"PID: Kp={info['pid_gains']['kp']:.2f} "
          f"Ki={info['pid_gains']['ki']:.2f} "
          f"Kd={info['pid_gains']['kd']:.2f}")

print("-" * 80)
print(f"\n📊 Evaluation Summary:")
print(f"  Hit rate: {hits}/{n_eval_episodes} ({100*hits/n_eval_episodes:.1f}%)")
print(f"  Avg reward: {np.mean(total_rewards):.1f} ± {np.std(total_rewards):.1f}")
print(f"  Avg distance: {np.mean(distances):.1f}m ± {np.std(distances):.1f}m")
print(f"  Avg steps: {np.mean(steps_list):.1f} ± {np.std(steps_list):.1f}")
print(f"  Min distance: {np.min(distances):.1f}m")

## 11. Visualize Training Progress

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Load tensorboard logs
eval_log_path = os.path.join(run_dir, 'eval_logs', 'evaluations.npz')

if os.path.exists(eval_log_path):
    eval_data = np.load(eval_log_path)
    timesteps = eval_data['timesteps']
    results = eval_data['results']
    
    mean_rewards = results.mean(axis=1)
    std_rewards = results.std(axis=1)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(timesteps, mean_rewards, label='Mean Reward', linewidth=2)
    ax.fill_between(timesteps, 
                     mean_rewards - std_rewards, 
                     mean_rewards + std_rewards, 
                     alpha=0.3, label='± Std Dev')
    ax.set_xlabel('Timesteps', fontsize=12)
    ax.set_ylabel('Episode Reward', fontsize=12)
    ax.set_title(f'{ALGORITHM} Training Progress on {TARGET_MANEUVER.capitalize()} Target', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(run_dir, 'training_progress.png'), dpi=150)
    plt.show()
    
    print(f"✓ Training plot saved to {os.path.join(run_dir, 'training_progress.png')}")
else:
    print("⚠️ Evaluation logs not found")

## 12. Compare with Fixed PID Baseline

In [ ]:
# Test fixed PID baseline
baseline_env = MissilePIDEnv(
    map_size=10000.0,
    hit_radius=50.0,
    max_steps=500,
    dt=0.01,
    target_maneuver=TARGET_MANEUVER
)

baseline_hits = 0
baseline_distances = []

print("\nTesting Fixed PID Baseline (Kp=2.0, Ki=0.1, Kd=0.5)...\n")

for episode in range(n_eval_episodes):
    obs, info = baseline_env.reset()
    done = False
    
    # No RL - just keep PID gains fixed
    baseline_env.missile.set_pid_gains(2.0, 0.1, 0.5)
    
    while not done:
        # Zero action = no PID adjustment
        obs, reward, terminated, truncated, info = baseline_env.step(np.zeros(3))
        done = terminated or truncated
    
    if info['hit']:
        baseline_hits += 1
    baseline_distances.append(info['distance'])

print(f"\n📊 Comparison:")
print(f"  Fixed PID Hit Rate: {baseline_hits}/{n_eval_episodes} ({100*baseline_hits/n_eval_episodes:.1f}%)")
print(f"  RL Agent Hit Rate:  {hits}/{n_eval_episodes} ({100*hits/n_eval_episodes:.1f}%)")
print(f"  Improvement: {100*(hits - baseline_hits)/n_eval_episodes:+.1f}%")
print(f"\n  Fixed PID Avg Distance: {np.mean(baseline_distances):.1f}m")
print(f"  RL Agent Avg Distance:  {np.mean(distances):.1f}m")
print(f"  Improvement: {np.mean(baseline_distances) - np.mean(distances):.1f}m closer")

## 13. Download Trained Model

In [ ]:
# Create zip file of trained model
import shutil

archive_name = f"{run_name}_trained_model"
shutil.make_archive(
    f"/kaggle/working/{archive_name}",
    'zip',
    run_dir
)

print(f"✓ Model archived: /kaggle/working/{archive_name}.zip")
print(f"\n📥 Download from Kaggle Output panel")
print(f"\nContents:")
print(f"  - best_model/      (Best performing model)")
print(f"  - final_model.zip  (Final model after training)")
print(f"  - checkpoints/     (Periodic checkpoints)")
print(f"  - eval_logs/       (Evaluation metrics)")
print(f"  - tensorboard/     (Training logs)")

## 14. Summary

### What we did:
1. ✅ Set up GPU environment
2. ✅ Defined missile PID system
3. ✅ Created moving target with maneuvers
4. ✅ Built Gymnasium RL environment
5. ✅ Trained RL agent (PPO/SAC/TD3) on GPU
6. ✅ Evaluated performance
7. ✅ Compared with fixed PID baseline
8. ✅ Visualized training progress

### Next steps:
- Download trained model
- Test on different maneuvers
- Try other algorithms (change ALGORITHM variable)
- Adjust hyperparameters
- Train longer (increase TOTAL_TIMESTEPS)

### To use trained model locally:
```python
from stable_baselines3 import PPO  # or SAC, TD3
model = PPO.load('path/to/best_model.zip')
obs, info = env.reset()
action, _ = model.predict(obs, deterministic=True)
```